# Test

In [15]:
from ultralytics import YOLO
from autoencoder import DNIAnomalyDetector
import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from tqdm import tqdm
import os

def crop_image(image, bbox):
    """Crop image using bounding box coordinates."""
    x1, y1, x2, y2 = [int(coord) for coord in bbox]
    return image.crop((x1, y1, x2, y2))

def process_image(image_path, yolo_model, detector, loss_weights):
    """
    Process a single image: YOLO for cropping, then Autoencoder for validation.
    """
    # Usar YOLO solo para detectar y recortar
    results = yolo_model(image_path)[0]

    # Si no hay detecciones, usar la imagen original
    if len(results.boxes) == 0:
        return detector.predict(image_path, loss_weights=loss_weights)

    # Obtener la detección con mayor confianza para el recorte
    confidences = results.boxes.conf.cpu().numpy()
    best_idx = confidences.argmax()
    box = results.boxes[best_idx]

    # Recortar imagen usando el bbox
    original_image = Image.open(image_path)
    bbox = box.xyxy[0].cpu().numpy()
    cropped_image = crop_image(original_image, bbox)

    # Guardar temporalmente la imagen recortada
    temp_crop_path = f"temp_crop_{os.path.basename(image_path)}"
    try:
        cropped_image.save(temp_crop_path)
        # Validar con Autoencoder
        return detector.predict(temp_crop_path, loss_weights=loss_weights)
    finally:
        if os.path.exists(temp_crop_path):
            os.remove(temp_crop_path)

def test_detector(model_path, yolo_path, test_dir, anomaly_threshold=0.5, loss_weights={'mse': 1.0, 'ssim': 0.0}):
    """
    Test the DNI Anomaly Detector with YOLO preprocessing for cropping.
    """
    # Initialize models
    yolo_model = YOLO(yolo_path)
    detector = DNIAnomalyDetector()
    detector.load_model(model_path)
    
    # Print configuration
    print("\nModel Configuration:")
    print(f"Autoencoder Threshold: {detector.threshold:.6f}")
    print(f"Loss Weights: MSE={loss_weights['mse']}, SSIM={loss_weights['ssim']}")
    print(f"Anomaly Classification Threshold: {anomaly_threshold}")
    
    # Process test images
    test_path = Path(test_dir)
    test_images = list(test_path.glob('*.jpg'))
    
    results = []
    confidences = []
    
    print("\nTesting images...")
    with torch.no_grad():
        for img_path in tqdm(test_images, desc="Processing images"):
            try:
                confidence = process_image(str(img_path), yolo_model, detector, loss_weights)
                confidences.append(confidence)
                
                results.append({
                    'image': str(img_path),
                    'confidence': confidence,
                    'is_anomaly': confidence < anomaly_threshold
                })
                
                print(f"\nImage: {img_path.name}")
                print(f"Confidence: {confidence:.4f}")
                print(f"Is anomaly: {confidence < anomaly_threshold}")
                
            except Exception as e:
                print(f"\nError processing {img_path.name}: {str(e)}")
                continue
    
    if not confidences:
        print("\nNo images were processed!")
        return results
    
    # Calculate statistics
    confidences = np.array(confidences)
    
    print("\nConfidence Score Statistics:")
    print(f"Min confidence: {confidences.min():.4f}")
    print(f"Max confidence: {confidences.max():.4f}")
    print(f"Mean confidence: {confidences.mean():.4f}")
    print(f"Median confidence: {np.median(confidences):.4f}")
    print(f"Std confidence: {confidences.std():.4f}")
    
    # Print detailed results
    print("\nDetailed Results:")
    confidence_ranges = {
        'Very Low (0.0-0.2)': 0,
        'Low (0.2-0.4)': 0,
        'Medium (0.4-0.6)': 0,
        'High (0.6-0.8)': 0,
        'Very High (0.8-1.0)': 0
    }
    
    for result in results:
        conf = result['confidence']
        if conf < 0.2:
            confidence_ranges['Very Low (0.0-0.2)'] += 1
        elif conf < 0.4:
            confidence_ranges['Low (0.2-0.4)'] += 1
        elif conf < 0.6:
            confidence_ranges['Medium (0.4-0.6)'] += 1
        elif conf < 0.8:
            confidence_ranges['High (0.6-0.8)'] += 1
        else:
            confidence_ranges['Very High (0.8-1.0)'] += 1
    
    for range_name, count in confidence_ranges.items():
        print(f"{range_name}: {count} images ({(count/len(results))*100:.1f}%)")
    
    # Print classification summary
    anomalies = sum(1 for r in results if r['is_anomaly'])
    print(f"\nClassification Summary:")
    print(f"Total images tested: {len(results)}")
    print(f"Anomalies detected: {anomalies} ({(anomalies/len(results))*100:.1f}%)")
    print(f"Normal images: {len(results) - anomalies} ({((len(results)-anomalies)/len(results))*100:.1f}%)")
    
    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(confidences, bins=50, edgecolor='black')
    plt.title('Distribution of Confidence Scores')
    plt.xlabel('Confidence Score')
    plt.ylabel('Number of Images')
    plt.axvline(x=anomaly_threshold, color='r', linestyle='--', label=f'Anomaly Threshold ({anomaly_threshold})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('confidence_distribution.png')
    plt.close()
    
    return results

# Example usage
if __name__ == "__main__":
    model_path = "api/dni_anomaly_detector.pt"
    yolo_path = "YOLODataset_seg/runs/segment/train2/weights/best.pt"
    test_dir = "test/valid"
    
    results = test_detector(
        model_path=model_path,
        yolo_path=yolo_path,
        test_dir=test_dir,
        anomaly_threshold=0.85,
        loss_weights={'mse': 1.0, 'ssim': 0.0}
    )


Model Configuration:
Autoencoder Threshold: 0.007056
Loss Weights: MSE=1.0, SSIM=0.0
Anomaly Classification Threshold: 0.85

Testing images...


Processing images:   0%|          | 0/62 [00:00<?, ?it/s]


image 1/1 /home/francobertoldi/Documents/DNI-classifier/test/valid/0e11d788-bbdd-4afa-9e12-455462d52c44.jpg: 480x640 1 FrenteValido, 3.9ms
Speed: 1.2ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)


Processing images:   2%|▏         | 1/62 [00:00<00:17,  3.59it/s]


Image: 0e11d788-bbdd-4afa-9e12-455462d52c44.jpg
Confidence: 0.2882
Is anomaly: True

image 1/1 /home/francobertoldi/Documents/DNI-classifier/test/valid/0fa71614-d9c4-42f3-9088-7555e837d493.jpg: 480x640 1 FrenteValido, 3.7ms
Speed: 1.2ms preprocess, 3.7ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)


...


Image: 0f0e576d-1fc4-40cd-bd1e-1730b30bde71.jpg
Confidence: 0.2938
Is anomaly: True

image 1/1 /home/francobertoldi/Documents/DNI-classifier/test/valid/0ef79580-d5aa-49fb-9e60-1dc83e664001.jpg: 480x640 1 FrenteValido, 3.9ms
Speed: 1.3ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)


Processing images:  97%|█████████▋| 60/62 [00:11<00:00,  4.80it/s]


Image: 0ef79580-d5aa-49fb-9e60-1dc83e664001.jpg
Confidence: 0.0000
Is anomaly: True

image 1/1 /home/francobertoldi/Documents/DNI-classifier/test/valid/0ec39358-8af7-462a-b677-da2ae95048fe.jpg: 416x640 1 FrenteValido, 6.8ms
Speed: 1.8ms preprocess, 6.8ms inference, 5.4ms postprocess per image at shape (1, 3, 416, 640)

Image: 0ec39358-8af7-462a-b677-da2ae95048fe.jpg
Confidence: 0.2873
Is anomaly: True

image 1/1 /home/francobertoldi/Documents/DNI-classifier/test/valid/0e2ac685-526b-442d-a33f-c5e6f58e249e.jpg: 640x480 1 FrenteValido, 3.9ms
Speed: 0.7ms preprocess, 3.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


Processing images: 100%|██████████| 62/62 [00:11<00:00,  5.24it/s]


Image: 0e2ac685-526b-442d-a33f-c5e6f58e249e.jpg
Confidence: 0.1664
Is anomaly: True

Confidence Score Statistics:
Min confidence: 0.0000
Max confidence: 0.6677
Mean confidence: 0.1239
Median confidence: 0.0165
Std confidence: 0.1702

Detailed Results:
Very Low (0.0-0.2): 43 images (69.4%)
Low (0.2-0.4): 16 images (25.8%)
Medium (0.4-0.6): 2 images (3.2%)
High (0.6-0.8): 1 images (1.6%)
Very High (0.8-1.0): 0 images (0.0%)

Classification Summary:
Total images tested: 62
Anomalies detected: 62 (100.0%)
Normal images: 0 (0.0%)
